In [ ]:
import os
import numpy as np
import pickle 
import audiosegment 
from helper import *
from features import *


In [ ]:
def calculate_features(frames, freq, options):
    window_sec = 0.2
    window_n = int(freq * window_sec)

    st_f = stFeatureExtraction(frames, freq, window_n, window_n / 2)

    if st_f.shape[1] > 2:
        i0 = 1
        i1 = st_f.shape[1] - 1
        if i1 - i0 < 1:
            i1 = i0 + 1
        
        deriv_st_f = np.zeros((st_f.shape[0], i1 - i0), dtype=float)
        for i in range(i0, i1):
            i_left = i - 1
            i_right = i + 1
            deriv_st_f[:st_f.shape[0], i - i0] = st_f[:, i]
        return deriv_st_f
    elif st_f.shape[1] == 2:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f
    else:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f
        

In [ ]:
framerate = 16000
data_path = '/IEMOCAP_full_release/'
with open(data_path + 'data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [ ]:
speech = [s['signal'] for s in data2] 

In [ ]:
voiced_feat = []
duration = 0.1
threshold = 0.001

In [ ]:

for i in range(len(speech)):
    x_head = data2[i]['signal']
    #x_head = speech[i]
    seg = audiosegment.from_numpy_array(speech[i], framerate)
    seg = seg.filter_silence(duration_s=duration, threshold_percentage=threshold)
    st_features = calculate_features(seg.to_numpy_array(), framerate, None)
    #st_features = calculate_features(x_head, framerate, None)
    st_features, _ = pad_sequence_into_array(st_features, maxlen=100)
    voiced_feat.append(st_features.T)
    print(i)

In [ ]:
voiced_feat = np.array(voiced_feat)
voiced_feat.shape
np.save('speech.npy', voiced_feat)